In [ ]:
## CE 295 - Energy Systems and Control
# HW 2 : State Estimation in Oil Well Drilling
# Oski Bear, SID 18681868
# Prof. Moura
# Due Date is written here

# BEAR_OSKI_HW2.ipynb

import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from scipy import interp
from scipy import signal
%matplotlib inline
from __future__ import division
import pandas as pd
import control # Read http://python-control.sourceforge.net/manual/

fs = 15 # Font Size for plots

# Drill String Parameters

J_T =   # Table/top rotational inertia
J_B =    # Bottom/bit rotational inertia
k =       # Spring constant
b =      # Drag coefficient

In [ ]:
## Problem 2 - Observability Analysis

# State space matrices
A4 = np.matrix([[ , , , ], [ , , , ], [ , , , ], [ , , , ]])
B4 = 
C4 = 

# Compute observability Matrix for 4-state system and rank
O4 = 
print('Rank of Observability Matrix for four-state system')
print(np.linalg.matrix_rank(O4))

# New A Matrix, for 3-state system
A = 
B = 
C = 

# Observability Matrix for 3-state system and rank
O = 
print('Rank of Observability Matrix for three-state system')
print(np.linalg.matrix_rank(O))

In [ ]:
## Load Data
data=np.asarray(pd.read_csv("HW2_Data.csv",header=None))

t = data[:,0]      # t   : time vector [sec]
y_m = data[:,1]    # y_m : measured table velocity [radians/sec]
Torq = data[:,2]   # Torq: table torque [N-m]
omega_B_true = data[:,3]    # \omega_B : true rotational speed of bit [radians/sec]

# Plot Data
plt.figure(num=1, figsize=(8, 9), dpi=80, facecolor='w', edgecolor='k')

plt.subplot(2,1,1)
plt.plot( )
# Plot table torque

plt.subplot(2,1,2)
plt.plot( )
# Plot measured table velocity

plt.show()

In [ ]:
## Problem 4 - Luenberger Observer

# Eigenvalues of open-loop system
print('Eigenvalues of open-loop system:')
lam_A,evec = 
print(lam_A)

# Desired poles of estimation error system
#   They should have negative real parts
#   Complex conjugate pairs
lam_luen = [ , , ]

# Compute observer gain (See Remark 3.1 in Notes. Use "acker" command)
L = 

# State-space Matrices for Luenberger Observer
A_lobs = 
B_lobs = 
C_lobs = 
D_lobs = 

sys_lobs = signal.lti( , , , )

# Inputs to observer
u = [] 

# Initial Conditions
x_hat0 = [ , , ]

# Simulate Response
tsim, y, x_hat = signal.lsim(sys_lobs, U=u, T=t, X0=x_hat0)

# Parse states
theta_hat = x_hat[:,0]
omega_T_hat = 
omega_B_hat = 

# Plot Results
plt.figure(num=1, figsize=(8, 9), dpi=80, facecolor='w', edgecolor='k')

plt.subplot(2,1,1)
# Plot true and estimated bit velocity

plt.subplot(2,1,2)
# Plot error between true and estimated bit velocity


In [ ]:
## Problem 5 - Kalman Filter
# Noise Covariances
W =  #You design this one. 
N = 
Sig0 = 

# Initial Condition
x_hat0 = [ ,  ,  ]
states0 = np.r_[x_hat0, np.squeeze(np.asarray(Sig0.reshape(9,1)))]

# Ordinary Differential Equation for Kalman Filter
def ode_kf(z,it):
    
    # Parse States
    x_hat = np.matrix(z[:3]).T
    Sig = np.matrix((z[3:]).reshape(3,3))
    
    # Interpolate input signal data
    iTorq = interp(it, t, Torq)
    iy_m = interp(it, t, y_m)
    
    # Compute Kalman Gain
    L = 
    
    # Kalman Filter
    x_hat_dot = 
    
    # Riccati Equation
    Sig_dot = 
    
    # Concatenate LHS
    z_dot = np.r_[x_hat_dot, Sig_dot.reshape(9,1)]
    
    return(np.squeeze(np.asarray(z_dot)))


# Integrate Kalman Filter ODEs
z = odeint(ode_kf, states0, t)

# Parse States
theta_hat = z[:,0]
omega_T_hat = 
omega_B_hat = 
Sig33 =      # Parse out the (3,3) element of Sigma only!

omega_B_tilde = omega_B_true - omega_B_hat
omega_B_hat_upperbound = omega_B_hat + np.sqrt(Sig33)
omega_B_hat_lowerbound = omega_B_hat - np.sqrt(Sig33)

RMSE = np.sqrt(np.mean(np.power(omega_B_tilde,2)))
print('Kalman Filter RMSE: ' + str(RMSE) + ' rad/s')


# Plot Results
plt.figure(num=3, figsize=(8, 9), dpi=80, facecolor='w', edgecolor='k')

plt.subplot(2,1,1)
#   Plot true and estimated bit velocity
#   Plot estimated bit velocity plus/minus one sigma

plt.subplot(2,1,2)
#   Plot error between true and estimated bit velocity

plt.show()

In [ ]:
## Problem 6 - Extended Kalman Filter

# New nonlinear spring parameters
k1 = 
k2 = 

# Noise Covariances
W =  #You design this one. 
N = 
Sig0 = 

# Initial Condition
x_hat0 = [ ,  ,  ]
states0 = np.r_[x_hat0, np.squeeze(np.asarray(Sig0.reshape(9,1)))]

# Ordinary Differential Equation for Kalman Filter
def ode_ekf(z,it):
    
    # Parse States
    theta_hat = z[0]
    omega_T_hat = z[1]
    omega_B_hat = z[2]
    Sig = np.matrix((z[3:]).reshape(3,3))
    
    # Interpolate input signal data
    iTorq = interp(it, t, Torq)
    iy_m = interp(it, t, y_m)
    
    # Compute Jacobians
    F = # YOU DERIVE THESE
    H = # YOU DERIVE THESE
    
    # Compute Kalman Gain
    L = 
    
    # Compute EKF system matrices
    y_hat = omega_T_hat
    
    theta_hat_dot = 
    omega_T_hat_dot = 
    omega_B_hat_dot = 
    
    # Riccati Equation
    Sig_dot = 
    
    # Concatenate LHS
    z_dot = np.r_[theta_hat_dot, omega_T_hat_dot, omega_B_hat_dot, Sig_dot.reshape(9,1)]
    
    return(np.squeeze(np.asarray(z_dot)))


# Integrate Extended Kalman Filter ODEs
z = odeint(ode_ekf, states0, t)

# Parse States
theta_hat = z[:,0]
omega_T_hat = 
omega_B_hat = 
Sig33 = 

omega_B_tilde = omega_B_true - omega_B_hat
omega_B_hat_upperbound = omega_B_hat + np.sqrt(Sig33)
omega_B_hat_lowerbound = omega_B_hat - np.sqrt(Sig33)

RMSE = np.sqrt(np.mean(np.power(omega_B_tilde,2)))
print('Extended Kalman Filter RMSE: ' + str(RMSE) + ' rad/s')


# Plot Results
plt.figure(num=3, figsize=(8, 9), dpi=80, facecolor='w', edgecolor='k')

plt.subplot(2,1,1)
#   Plot true and estimated bit velocity
#   Plot estimated bit velocity plus/minus one sigma

plt.subplot(2,1,2)
#   Plot error between true and estimated bit velocity

plt.show()